In [169]:
import pandas as pd
import numpy as np
import plotly as px
import matplotlib.pyplot as plt
from colorama import Fore, Back, Style
import json
import os
import matplotlib.colors as mcolors

# Etape I --> Importer / Nettoyer

### Importation

In [170]:
wd = os.getcwd()

In [222]:
file_path = '../data/original/ESP_PUBLIC.IDENTITE_ARBRE.csv'
df = pd.read_csv(file_path, sep=',', header = 0, index_col=False)

In [223]:
df.head()

,ELEM_POINT_ID,CODE,NOM,GENRE,GENRE_DESC,CATEGORIE,CATEGORIE_DESC,SOUS_CATEGORIE,SOUS_CATEGORIE_DESC,CODE_PARENT,...,COURRIER,IDENTIFIANTPLU,TYPEIMPLANTATIONPLU,INTITULEPROTECTIONPLU,ANNEEABATTAGE,ESSOUCHEMENT,DIAMETREARBRE,CAUSEABATTAGE,COLLECTIVITE,GeoJSON
0,43866,ESP40237,ESP40237,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7209090239891..."
1,38181,ESP38129,ESP38129,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP1075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7048112482231..."
2,38182,ESP38130,ESP38130,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP1075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.704875754357,..."
3,38183,ESP38131,ESP38131,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP1075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7049488370901..."
4,38184,ESP38132,ESP38132,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP1075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7050118332264..."


### Infos

In [224]:
df.shape

(31618, 32)

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31618 entries, 0 to 31617
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ELEM_POINT_ID          31618 non-null  int64  
 1   CODE                   31618 non-null  object 
 2   NOM                    31618 non-null  object 
 3   GENRE                  31618 non-null  object 
 4   GENRE_DESC             31618 non-null  object 
 5   CATEGORIE              31618 non-null  object 
 6   CATEGORIE_DESC         31618 non-null  object 
 7   SOUS_CATEGORIE         31618 non-null  object 
 8   SOUS_CATEGORIE_DESC    31618 non-null  object 
 9   CODE_PARENT            31618 non-null  object 
 10  CODE_PARENT_DESC       31618 non-null  object 
 11  ADR_SECTEUR            31618 non-null  int64  
 12  BIEN_REFERENCE         31618 non-null  object 
 13  GENRE_BOTA             30685 non-null  object 
 14  ESPECE                 29084 non-null  object 
 15  VA

### Cleaning empty or almost empty

In [226]:
nullThreshold = len(df["ELEM_POINT_ID"])*50/100
nullThreshold

15809.0

In [227]:
df["ELEM_POINT_ID"].count()

31618

#### Utils

In [228]:
def is_suppressed(df, colName):
    nullNum = df[colName].count()
    if nullNum > nullThreshold:
        answer =  False
    else:
        answer =  True
    return answer

In [229]:
def has_more_than_one(df,col):
    value = df[col].nunique()
    if value > 1:
        return True
    else:
        return False

In [230]:
def drop_column_threshold(df):
    colSuppressed = []
    dff = df.copy()
    for col in df.columns:
        if is_suppressed(df, col):
            dff.drop(columns=col, inplace=True)
            colSuppressed.append(col)
            print(f"col suppressed : {col}")
            
    return dff, colSuppressed

In [231]:
def clean_unique_value(df):
    dff = df.copy()
    colSuppressed = []
    for col in df.columns:
        if not has_more_than_one(df,col):
            dff.drop(columns=col, inplace=True)
            colSuppressed.append(col)
            print(f"col suppressed : {col}")
    return dff, colSuppressed

In [240]:
def has_only_different_value(df,col):
    if len(df[col]) == len(df[col].value_counts()):
        return True
    else:
        return False

In [268]:
def clean_only_diff_value(df):
    dff = df.copy()
    colSuppressed = []
    for col in dff.columns:
        if has_only_different_value(dff,col) and col != "GeoJSON":
            dff.drop(columns=col, inplace=True)
            colSuppressed.append(col)
            print(f"col suppressed : {col}")
            
    return dff, colSuppressed      

#### Cleaning

In [287]:
df_clean = df.copy()

In [288]:
df_clean, colSuppressed = drop_column_threshold(df)
print("-------------------------------------------------")
print(f"Columns suppressed => {len(colSuppressed)}")
print("")

-------------------------------------------------
Columns suppressed => 0



In [289]:
df_clean, colSuppressed = clean_unique_value(df_clean)
print("-------------------------------------------------")
print(f"Columns suppressed => {len(colSuppressed)}")
print("")

-------------------------------------------------
Columns suppressed => 0



In [290]:
df_clean, colSuppressed = clean_only_diff_value(df_clean)
print("-------------------------------------------------")
print(f"Columns suppressed => {len(colSuppressed)}")
print("")

col suppressed : ELEM_POINT_ID
col suppressed : CODE
col suppressed : NOM
col suppressed : BIEN_REFERENCE
-------------------------------------------------
Columns suppressed => 4



In [291]:
df_clean.columns

Index(['SOUS_CATEGORIE', 'SOUS_CATEGORIE_DESC', 'CODE_PARENT',
       'CODE_PARENT_DESC', 'ADR_SECTEUR', 'GENRE_BOTA', 'ESPECE',
       'STADEDEDEVELOPPEMENT', 'ANNEEDEPLANTATION', 'COLLECTIVITE', 'GeoJSON'],
      dtype='object')

### Clean Tween columns

In [292]:
df_clean.drop(columns="SOUS_CATEGORIE", inplace=True)
df_clean.drop(columns="CODE_PARENT", inplace=True)

### Parse GeoJson

In [313]:
def split_GeoJSON(df):
    dff = df.copy()
    dff['coordinates'] = dff['GeoJSON'].apply(lambda x: json.loads(x)['coordinates'])
    dff['latitude'] = dff['coordinates'].apply(lambda x: x[1])
    dff['longitude'] = dff['coordinates'].apply(lambda x: x[0])
    dff['sum_coordinate'] = dff['latitude'] + dff['longitude']
    return dff

In [314]:
df_clean = split_GeoJSON(df_clean)

# First Analysis

In [315]:
df = df_clean.copy()
df.head()

,SOUS_CATEGORIE_DESC,CODE_PARENT_DESC,ADR_SECTEUR,GENRE_BOTA,ESPECE,STADEDEDEVELOPPEMENT,ANNEEDEPLANTATION,COLLECTIVITE,GeoJSON,coordinates,latitude,longitude,sum_coordinate
0,Arbre de voirie,R Andrieux PM Curie R François,6,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7209090239891...","[5.72090902398911, 45.1623384592357]",45.162338,5.720909,50.883247
1,Arbre de voirie,Av Rh Danube Vallier E Claires,3,Platanus,hispanica,Arbre adulte,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7048112482231...","[5.70481124822311, 45.1779382313614]",45.177938,5.704811,50.882749
2,Arbre de voirie,Av Rh Danube Vallier E Claires,3,Platanus,hispanica,Arbre adulte,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.704875754357,...","[5.704875754357, 45.1779321250833]",45.177932,5.704876,50.882808
3,Arbre de voirie,Av Rh Danube Vallier E Claires,3,Platanus,hispanica,Arbre adulte,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7049488370901...","[5.70494883709017, 45.1779361272749]",45.177936,5.704949,50.882885
4,Arbre de voirie,Av Rh Danube Vallier E Claires,3,Platanus,hispanica,Arbre adulte,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7050118332264...","[5.70501183322648, 45.1779323479383]",45.177932,5.705012,50.882944


In [294]:
year = df["ANNEEDEPLANTATION"]

In [306]:
df.loc[5,"GeoJSON"]

'{"type":"Point","coordinates":[5.70507321129802,45.1779286067872]}'

In [310]:
import json
import pandas as pd

# create example dataframe
df = pd.DataFrame({'value': ['{"type":"Point","coordinates":[5.70507321129802,45.1779286067872]}']})

# extract latitude and longitude using json.loads()
df['coordinates'] = df['value'].apply(lambda x: json.loads(x)['coordinates'])
df['latitude'] = df['coordinates'].apply(lambda x: x[1])
df['longitude'] = df['coordinates'].apply(lambda x: x[0])

# print resulting dataframe
df['longitude']
df.loc[0,'latitude']

45.1779286067872